In [1]:
# Gerekli kütüphanelerin içe aktarılması
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
from scipy.io.wavfile import write
from datetime import datetime
import os
import numpy as np
import speech_recognition as sr
import threading
import serial 
import time

# Arduino bağlantı bilgileri
ARDUINO_PORT = "COM8"  # Senin Arduino'n hangi porta bağlıysa onu yaz (örneğin: COM4)
BAUD_RATE = 9600

def arduinoya_komut_gonder(metin):
    try:
        arduino = serial.Serial(ARDUINO_PORT, BAUD_RATE)
        time.sleep(2)  # Bağlantı otursun
        if "yak" in metin:
            arduino.write(b"LED_YAK\n")
            sonuc = "💡 LED yak komutu gönderildi."
        elif "söndür" in metin or "sondur" in metin:
            arduino.write(b"LED_SONDUR\n")
            sonuc = "🕯️ LED söndür komutu gönderildi."
        else:
            sonuc = f"❗ Komut algılanamadı: \"{metin}\""

        arduino.close()
        return sonuc
    except Exception as e:
        return f"⚠️ Arduino bağlantı hatası: {e}"

# Kayıt ve metin klasörlerinin tanımlanması
KAYIT_KLASORU = "C:/Users/beste/Desktop/bitirme/Ses Dosyaları"
METIN_KLASORU = "C:/Users/beste/Desktop/bitirme/Metin Dosyaları"
os.makedirs(KAYIT_KLASORU, exist_ok=True)
os.makedirs(METIN_KLASORU, exist_ok=True)

# Dosya isimleri için zaman damgası üretir
def zaman_damgasi():
    return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# WAV dosyasını metne dönüştürür ve .txt olarak kaydeder
def wav_to_text(dosya_yolu):
    r = sr.Recognizer()
    try:
        with sr.AudioFile(dosya_yolu) as source:
            audio = r.record(source)
        text = r.recognize_google(audio, language="tr-TR")
        txt_adi = os.path.splitext(os.path.basename(dosya_yolu))[0] + ".txt"
        txt_yolu = os.path.join(METIN_KLASORU, txt_adi)
        with open(txt_yolu, 'w', encoding='utf-8') as f:
            f.write(text)
        return text, txt_yolu
    except sr.UnknownValueError:
        return "❌ Ses anlaşılamadı.", None
    except sr.RequestError as e:
        return f"🌐 Google servisine ulaşılamadı: {e}", None

# Ses klasörünü açar
def ses_klasoru_ac():
    try:
        os.startfile(KAYIT_KLASORU)
    except Exception as e:
        messagebox.showerror("Hata", f"Ses klasörü açılamadı:\n{e}")

# Metin klasörünü açar
def metin_klasoru_ac():
    try:
        os.startfile(METIN_KLASORU)
    except Exception as e:
        messagebox.showerror("Hata", f"Metin klasörü açılamadı:\n{e}")

# Kayıt geçmişini listbox içinde günceller
def guncelle_gecmis():
    gecmis_listbox.delete(0, tk.END)
    kayitlar = sorted(
        [f for f in os.listdir(KAYIT_KLASORU) if f.endswith(".wav")],
        reverse=True
    )
    for f in kayitlar:
        gecmis_listbox.insert(tk.END, f)
# Kaydı başlat butonuna basıldığında çalışan fonksiyon
def baslat_kayit():
    try:
        sure = int(sure_entry.get())
        if sure <= 0:
            messagebox.showwarning("Uyarı", "Süre 0'dan büyük olmalı.")
            return
    except ValueError:
        messagebox.showwarning("Uyarı", "Geçerli bir sayı girin.")
        return

    try:
        fs = int(sampling_rate_var.get())
        channels = int(channel_var.get())
    except ValueError:
        messagebox.showwarning("Uyarı", "Geçerli örnekleme ayarlarını seçin.")
        return

    etiket = secilen_etiket.get()

    # Kayıttan önceki 3 saniyelik geri sayım
    def geri_sayim(kalan):
        if kalan > 0:
            sonuc_label.config(text=f"⏳ Kayıt başlıyor: {kalan}")
            pencere.after(1000, geri_sayim, kalan - 1)
        else:
            kaydi_baslat_ve_sayac(sure, fs, channels, etiket)

    geri_sayim(3)
# Kayıt alma ve sayaç fonksiyonu
def kaydi_baslat_ve_sayac(sure, fs, channels, etiket):
    etiket = etiket.replace(" ", "_")
    dosya_adi = f"kayit_{etiket}_{zaman_damgasi()}.wav"
    dosya_yolu = os.path.join(KAYIT_KLASORU, dosya_adi)
    sonuc_label.config(text=f"🎙️ Kayıt alınıyor ({sure})")
    pencere.update()

    ses = np.empty((0, channels), dtype='int16')

    # Kayıt işlemi ayrı bir iş parçacığında çalışır
    def kaydet():
        nonlocal ses
        ses = sd.rec(int(sure * fs), samplerate=fs, channels=channels, dtype='int16')
        sd.wait()

    thread = threading.Thread(target=kaydet)
    thread.start()

    # Geriye doğru sayım fonksiyonu
    def sayac(kalan):
        if kalan >= 0:
            sonuc_label.config(text=f"🎙️ Kayıt alınıyor ({kalan})")
            pencere.after(1000, sayac, kalan - 1)
        else:
            thread.join()  # Kayıt bitene kadar bekle
            write(dosya_yolu, fs, ses)  # Ses dosyasını kaydet
            sonuc_label.config(text="🧠 Ses çözümleniyor...")
            pencere.update()
            text, txt_yolu = wav_to_text(dosya_yolu)
            if txt_yolu:
                sonuc_label.config(
                    text=f"✅ Metin çözümlendi:\n\n{text}\n\n📁 Metin: {txt_yolu}\n📁 Ses: {dosya_yolu}"
                )
            else:
                sonuc_label.config(text=text)
            guncelle_gecmis()

    sayac(sure)  # Sayaç başlatılır



def sesli_komut_kayit():
    fs = 44100
    sure = 4  
    sonuc_label.config(text="🎤 Komut alınıyor...")
    pencere.update()

    try:
        ses = sd.rec(int(sure * fs), samplerate=fs, channels=1, dtype='int16')
        sd.wait()
    except Exception as e:
        messagebox.showerror("Hata", f"Ses kaydı başarısız: {e}")
        return

    # Geçici olarak ses dosyasını kaydet
    gecici_dosya = "gecici_komut.wav"
    write(gecici_dosya, fs, ses)

    sonuc_label.config(text="🧠 Komut analiz ediliyor...")
    pencere.update()

    text, _ = wav_to_text(gecici_dosya)
    sonuc = arduinoya_komut_gonder(text)

    sonuc_label.config(text=f"🎤 Komut: {text}\n🔁 {sonuc}")

    os.remove(gecici_dosya)


# ======================= ARAYÜZ OLUŞTURMA =======================

pencere = tk.Tk()
pencere.title("Ses Kaydı ve Metin Çözümleme")
pencere.geometry("900x900")
pencere.resizable(False, False)

# Süre girişi
tk.Label(pencere, text="🎙️ Kayıt Süresi (saniye):", font=("Arial", 12)).pack(pady=5)
sure_entry = tk.Entry(pencere, font=("Arial", 12), justify="center")
sure_entry.pack()
sure_entry.insert(0, "5")

# Etiket seçimi
etiketler = ["Komut", "Soru", "Yanıt", "Duygu", "Genel", "Test"]
secilen_etiket = tk.StringVar()
secilen_etiket.set(etiketler[0])
tk.Label(pencere, text="🏷️ Kategori Seçin:", font=("Arial", 12)).pack()
tk.OptionMenu(pencere, secilen_etiket, *etiketler).pack(pady=5)

# Örnekleme frekansı seçimi
tk.Label(pencere, text="🎛️ Örnekleme Frekansı (Hz):", font=("Arial", 12)).pack()
sampling_rate_var = tk.StringVar()
sampling_rate_var.set("16000")
tk.OptionMenu(pencere, sampling_rate_var, "8000", "16000", "22050", "44100", "48000").pack(pady=5)

# Kanal seçimi
tk.Label(pencere, text="🎚️ Kanal Sayısı:", font=("Arial", 12)).pack()
channel_var = tk.StringVar()
channel_var.set("1")
tk.OptionMenu(pencere, channel_var, "1", "2").pack(pady=5)

# Kaydı başlat butonu
tk.Button(pencere, text="Kaydı Başlat", command=baslat_kayit,
          font=("Arial", 12), bg="#4CAF50", fg="white", width=20).pack(pady=10)


frame = tk.Frame(pencere)
frame.pack(pady=5)
# Arduino' yu kontrol etme butonu
tk.Button(pencere, text="🎤 Sesli Komutla Arduino’yu Kontrol Et", command=sesli_komut_kayit,
          font=("Arial", 12), bg="#f44336", fg="white", width=40).pack(pady=10)
# Klasör açma butonları
tk.Button(frame, text="🎵 Ses Kayıtlarını Aç", command=ses_klasoru_ac,
          font=("Arial", 10), bg="#2196F3", fg="white", width=22).pack(side="left", padx=5)

tk.Button(frame, text="📄 Metin Dosyalarını Aç", command=metin_klasoru_ac,
          font=("Arial", 10), bg="#9C27B0", fg="white", width=22).pack(side="left", padx=5)

# Sonuçları gösteren etiket
sonuc_label = tk.Label(pencere, text="", wraplength=700, justify="left", font=("Arial", 10))
sonuc_label.pack(pady=10)

# Geçmiş listesi
tk.Label(pencere, text="📜 Kayıt Geçmişi:", font=("Arial", 12)).pack(pady=5)
gecmis_listbox = tk.Listbox(pencere, width=90, height=8, font=("Courier", 9))
gecmis_listbox.pack()



# Geçmişi yenile butonu
tk.Button(pencere, text="🔄 Geçmişi Yenile", command=guncelle_gecmis,
          font=("Arial", 10), bg="#FF9800", fg="white", width=20).pack(pady=5)


# İlk geçmiş yüklemesi
guncelle_gecmis()

# Arayüzü başlat
pencere.mainloop()
